In [151]:
import pandas as pd
from streamlit_assets.components import get_recos_users
from sklearn.preprocessing import LabelEncoder
from surprise import SVD, NMF, KNNBasic, SVDpp
from surprise.model_selection import cross_validate
from surprise import Dataset
from surprise import Reader
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


### Exploring metacritic data for recos

In [11]:
meta_df = pd.read_csv('data/metacritic.csv').dropna(subset='critic_meta_score')
#meta_df = meta_df.rename(columns={"title":"item_id"})

meta_df.head()


,title,rank,critic_meta_score,review_content,review_source,author,review_date,summary,meta_score,user_score,release_date,link,critics_reviews_link
0,Rectify: Season 4,1,100.0,It allows us to know and care for these charac...,Collider,Allison Keene,"Oct 27, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
1,Rectify: Season 4,1,100.0,"Rectify, a drama entering its final season on ...",The New York Times,James Poniewozik,"Oct 25, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
2,Rectify: Season 4,1,100.0,No other series so poignantly probes the human...,Salon,Melanie McFarland,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
3,Rectify: Season 4,1,100.0,None of these characters is particularly happy...,Yahoo TV,Ken Tucker,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
4,Rectify: Season 4,1,100.0,Rectify is the best series I have ever seen on...,The Daily Beast,Malcolm Jones,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...


In [212]:
meta_df['tv_show'] = meta_df['title'].str.split(': Season').str[0]
meta_df.head()

,title,rank,critic_meta_score,review_content,review_source,author,review_date,summary,meta_score,user_score,release_date,link,critics_reviews_link,title_group,user_id,item_id,rating,tv_show
0,Rectify: Season 4,1,100.0,It allows us to know and care for these charac...,Collider,Allison Keene,"Oct 27, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...,Rectify,58,Rectify: Season 4,100,Rectify
1,Rectify: Season 4,1,100.0,"Rectify, a drama entering its final season on ...",The New York Times,James Poniewozik,"Oct 25, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...,Rectify,576,Rectify: Season 4,100,Rectify
2,Rectify: Season 4,1,100.0,No other series so poignantly probes the human...,Salon,Melanie McFarland,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...,Rectify,1025,Rectify: Season 4,100,Rectify
3,Rectify: Season 4,1,100.0,None of these characters is particularly happy...,Yahoo TV,Ken Tucker,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...,Rectify,803,Rectify: Season 4,100,Rectify
4,Rectify: Season 4,1,100.0,Rectify is the best series I have ever seen on...,The Daily Beast,Malcolm Jones,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...,Rectify,945,Rectify: Season 4,100,Rectify


In [213]:
id_maker = LabelEncoder()
meta_df['user_id'] = id_maker.fit_transform(meta_df['author'])
meta_df['item_id'] = meta_df['title']
meta_df['rating'] = meta_df['critic_meta_score'].apply(int)

reviews = meta_df[['user_id','item_id','rating']]

In [214]:
tv_list = ['Breaking Bad: Season 5',
           'Rectify: Season 4',
           'Better Call Saul: Season 5',
           'The Leftovers: Season 1',
           'Lodge 49',
           'The Underground Railroad',
           'Master of None: Season 1',
           'Normal People: Season 1',
           'Fleabag: Season 1'
          ]


In [215]:
df_recos = get_recos_users(tv_list,reviews, filter_items=True,filter_n=10, default_rating=90)

C:\git\reco_system\streamlit_assets\components.py:82: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [216]:
df_merged = df_recos.merge(meta_df, on='item_id').drop_duplicates(subset='item_id')
df_merged['difference'] = df_merged['user match'] - df_merged['meta_score']
df_merged[['title','summary','user match','meta_score','rank','difference']].head(30).sort_values(by='user match', ascending = False)

,title,summary,user match,meta_score,rank,difference
429,Better Call Saul: Season 6,The sixth and final season of the Breaking Bad...,95.024701,94,35,1.024701
128,Atlanta: Season 2,The second season subtitled the Atlanta Robbin...,94.147231,97,15,-2.852769
567,The Americans: Season 5,The espionage drama was renewed for two season...,93.101516,94,42,-0.898484
56,The Leftovers: Season 3,The third and final season of the Damon Lindel...,93.085462,98,9,-4.914538
348,The Americans: Season 4,Elizabeth and Phillips are given a new dangero...,92.821630,95,31,-2.178370
492,Barry: Season 3,Barry finds it's not just other people stoppin...,92.410778,94,38,-1.589222
294,Fleabag: Season 2,A priest (Andrew Scott) helps Fleabag change t...,91.724099,96,25,-4.275901
410,Better Things: Season 3,Co-creator/writer/star Pamela Adlon will again...,91.413562,94,34,-2.586438
73,The Wire: Season 3,In chronicling a multi-generational family bus...,90.711804,98,10,-7.288196
84,The Wire: Season 4,One of the most heralded and unique dramas on ...,90.282888,98,11,-7.717112


In [218]:
df_grouped = df_merged.groupby('tv_show').mean()

In [219]:
df_grouped[['user match','meta_score','difference','rank']].sort_values(by='user match', ascending = False).head(20)

,user match,meta_score,difference,rank
tv_show,,,,
It's a Sin,92.746325,91.000000,1.746325,77.000000
City So Real,92.247868,93.000000,-0.752132,45.000000
The Beatles: Get Back,92.149175,85.000000,7.149175,234.000000
Fleabag,91.724099,96.000000,-4.275901,25.000000
Severance,91.258447,83.000000,8.258447,350.000000
Homecoming: A Film by Beyoncé,90.407554,93.000000,-2.592446,50.000000
Unbelievable,90.381903,83.000000,7.381903,368.000000
Atlanta,90.342956,92.666667,-2.323711,60.333333
Frozen Planet,89.932915,91.000000,-1.067085,84.000000


### Now, looking at trends between Metacritic rank and our SVD recommendation scores

In [220]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Season by Season data","Grouped by TV Show Data"))

fig.append_trace(list(px.scatter(df_merged, x='rank', y='user match',trendline = "ols",trendline_color_override="red", height = 350).select_traces())[0],
                           row=1, col=1)
fig.append_trace(list(px.scatter(df_merged, x='rank', y='user match',trendline = "ols",trendline_color_override="red", height = 350).select_traces())[1],
                           row=1, col=1)
fig.append_trace(list(px.scatter(df_grouped, x='rank', y='user match',trendline = "ols",trendline_color_override="red", height = 350).select_traces())[0],
                           row=1, col=2)
fig.append_trace(list(px.scatter(df_grouped, x='rank', y='user match',trendline = "ols",trendline_color_override="red", height = 350).select_traces())[1],
                           row=1, col=2)
fig.show()

### What do we see?
- The Score is very proportional to the rank, which is not great because we would want recommendations not be only a mere reflection of the meta score/rank, but also to capture an idiosyncratic component based on our preferences
- However, for extreme values, and most interestingly for high ranks, which are the most likely recommendations, the linear relationship seems to break down


In [225]:
df_merged_top = df_merged.sort_values(by='meta_score',ascending=False).reset_index().head(100) #top 100 seasons
df_grouped_top = df_grouped.sort_values(by='meta_score',ascending=False).reset_index().head(100) #top 100 shows

In [227]:
fig2 = make_subplots(rows=1, cols=2, subplot_titles=("Top 100 Seasons","Top 100 Shows"))

fig2.append_trace(list(px.scatter(df_merged_top, x='rank', y='user match',trendline = "ols",trendline_color_override="red", height = 350).select_traces())[0],
                           row=1, col=1)
fig2.append_trace(list(px.scatter(df_merged_top, x='rank', y='user match',trendline = "ols",trendline_color_override="red", height = 350).select_traces())[1],
                           row=1, col=1)
fig2.append_trace(list(px.scatter(df_grouped_top, x='rank', y='user match',trendline = "ols",trendline_color_override="red", height = 350).select_traces())[0],
                           row=1, col=2)
fig2.append_trace(list(px.scatter(df_grouped_top, x='rank', y='user match',trendline = "ols",trendline_color_override="red", height = 350).select_traces())[1],
                           row=1, col=2)
fig2.show()

### Much better! OLS R²'s have dropped from 0.9 to 0.2